In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time
from absl import app
from absl import flags
import numpy as np
import tensorflow as tf


from tensorflow.contrib.tpu.python.tpu import async_checkpoint
from tensorflow.contrib.training.python.training import evaluation
from tensorflow.core.protobuf import rewriter_config_pb2
from tensorflow.python.estimator import estimator
from tensorflow.python.keras import backend as K


import sys
sys.path.append('../train-final')
import main, imagenet_input
from main import final_model_fn


In [3]:

data_dir = "/Users/roxanefischer/Documents/cours/3A/Stage_ML/single-path-nas/other/tiny_imagenet/all"

FLAGS = flags.FLAGS
FLAGS(["data_format, input_image_size, num_label_classes, num_eval_images, eval_batch_size, num_train_images, train_batch_size, use_tpu, model_dir, log_step_count_steps, parse_search_dir"])
flags.FLAGS.data_format="channels_last"
flags.FLAGS.input_image_size = 64
flags.FLAGS.log_step_count_steps = 64
flags.FLAGS.num_label_classes =200
flags.FLAGS.num_eval_images=9832
flags.FLAGS.eval_batch_size=1024
flags.FLAGS.num_train_images=100000
flags.FLAGS.train_batch_size=1024
flags.FLAGS.use_tpu=False
flags.FLAGS.parse_search_dir = "/Users/roxanefischer/Desktop/tiny_imagenet_local/v4_full_l0.1_nw"
#TEST MAIN V1
flags.FLAGS.model_dir="/Users/roxanefischer/Downloads/final_model"

save_checkpoints_steps=10000000

In [4]:
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver('',zone=None, project=None)

config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=FLAGS.model_dir,
    save_checkpoints_steps=save_checkpoints_steps,
    log_step_count_steps=FLAGS.log_step_count_steps,
    session_config=tf.ConfigProto(
        graph_options=tf.GraphOptions(
            rewrite_options=rewriter_config_pb2.RewriterConfig(
                disable_meta_optimizer=True))),
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig
        .PER_HOST_V2))


params = dict(steps_per_epoch=FLAGS.num_train_images / FLAGS.train_batch_size)


model_est = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    model_fn=final_model_fn,
    config=config,
    train_batch_size=FLAGS.train_batch_size,
    eval_batch_size=FLAGS.eval_batch_size,
    export_to_tpu=FLAGS.export_to_tpu,
    params=params)

INFO:tensorflow:Using config: {'_model_dir': '/Users/roxanefischer/Downloads/final_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 10000000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': None, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': b'', '_evaluation_master': b'', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1251, num_shards=None, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_cluster': <tensorflow.contrib.cluster_

In [5]:
imagenet_train, imagenet_eval = [
    imagenet_input.ImageNetInput(
        is_training=is_training,
        data_dir=data_dir,
        transpose_input=True,
        cache=True and is_training,
        image_size=FLAGS.input_image_size,
        num_parallel_calls=64,
        use_bfloat16=False) for is_training in [True, False]
]

In [6]:
eval_steps = FLAGS.num_eval_images // FLAGS.eval_batch_size
def eval_results(ckpt ="/Users/roxanefischer/Desktop/models_tiny_imagenet_v2/model.ckpt-10000", eval_steps = eval_steps):
    eval_results = model_est.evaluate(
        input_fn=imagenet_eval.input_fn,
        steps=eval_steps,
        checkpoint_path=ckpt)
    return eval_results

train_steps = FLAGS.num_train_images // FLAGS.train_batch_size
def train_results(ckpt ="/Users/roxanefischer/Desktop/models_tiny_imagenet_v2/model.ckpt-10000", train_steps = train_steps):
    train_results = model_est.evaluate(
        input_fn=imagenet_train.input_fn,
        steps=train_steps,
        checkpoint_path=ckpt)
    return train_results

# Final model (350 epochs) : overfitting :

## On train set
{'loss': 1.8640844,
 'top_1_accuracy': 0.7998047,
 'top_5_accuracy': 0.91308594,
 'global_step': 35000}

## On eval : 
{'loss': 7.81235,
 'top_1_accuracy': 0.0107421875,
 'top_5_accuracy': 0.028320312,
 'global_step': 35000}